# Check how much data of different types we have


## Imports

In [1]:
import os

import numpy as np
import pandas as pd


## Path info

In [2]:
# where to find stuff
root_path = '/media/sam/SamData/Mosquitoes/'
expr_log_path_full = os.path.join(root_path, 'experiment_log.xlsx')
# data_inventory_path_full = os.path.join(root_path, 'analysis', 'spike_detection_review_expr39-75.xlsx')
data_inventory_path_full = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTxmAxmcKGkBg4aunlsG7pGZsdzjI8id-0l8wpZZxjUWkeiAHSOXCUW-6aUR5gztOjtzCrvGOqaxUtl/pub?output=xlsx'

## Which data files do we think are usable?

In [3]:
# Files that were previously identified as good in experiments 1-38 (from get_spike_rate_wbf_data.ipynb)
data_files_old = ['19.1', '19.2', '19.4', '19.5', '19.6',
              '22.0', '22.1', '22.2', '22.4', '22.5', '22.6', '22.7', '22.8', '22.9', '22.12',
              '23.0', '23.1', '23.2', '23.3', '23.4', '23.5', '23.6', 
              '24.7', '24.8', 
              '26.1', '26.2', '26.3', 
              '28.0', '28.1', '28.2', '28.5', '28.6', '28.8', '28.9', 
              '29.7', '29.8',
              '32.1',
              '37.0', '37.1', '37.2', '37.3',
              '38.8', '38.9', '38.10', '38.11', '38.14', '38.15',
              '15.1', '15.2', '15.3', '15.6', '15.8', '15.10',
              '16.0', '16.1', 
              '17.9',
              '18.0', '18.1', '27.5', '30.0', '30.1', '30.2', '31.3', '31.4', '31.5', '31.6',
              '33.0', '33.1', '33.3', '33.4', '33.5', '34.1', '34.4', '34.5', '34.7', '35.0', 
              '35.1', '36.0', '36.3', '36.4',
    ]

# convert these to tuples, i guess?
data_file_ids = [(int(f.split('.')[0]), int(f.split('.')[1])) for f in data_files_old]

In [4]:
# for experiments 39-current, I'm trying to keep a google sheet giving the quality/status of each trial:
#   https://docs.google.com/spreadsheets/d/1Ss7OxrC5hHUlTz6C7DVUP1ujM5IX8uHGOTLbFJS7a2k/edit?usp=sharing 
# (the path for accessing it via pandas is in data_path_inventory_full)
inventory_df = pd.read_excel(data_inventory_path_full)

# let's assume that trials categorized as 'Good', 'Sort', or 'Reanalyze' are going to be usable
good_idx = inventory_df['Status'].isin(['Good', 'Sort', 'Reanalyze'])

# get the file ids for these trials
expr_nums_df = inventory_df['Experiment Number'][good_idx].str.split('_')
expr_nums_df = expr_nums_df.apply(lambda row: row[0])
expr_nums = expr_nums_df.values.tolist()

axo_nums = inventory_df['Axo Number'][good_idx].values.tolist()

data_file_ids_new = [(int(en), int(an)) for en, an in zip(expr_nums, axo_nums)]

# combine
data_file_ids.extend(data_file_ids_new)

## Get info for these data files

In [5]:
# load experiment log
log_df = pd.read_excel(expr_log_path_full)
log_df

,Day,Axo Num,Fly Num,Sex,Species,Target Muscle Type,Target Muscle,Electrode Num,Video Count,Notes
0,01_20240316,NaN,1,F,Aedes,NaN,NaN,NaN,4,No axoscope recordings
1,02_20240318,0.0,1,F,Aedes,power,NaN,NaN,3,NaN
2,02_20240318,1.0,1,F,Aedes,power,NaN,NaN,3,NaN
3,02_20240318,2.0,1,F,Aedes,b1,NaN,NaN,0,NaN
4,02_20240318,3.0,1,F,Aedes,b1,NaN,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...
560,83_20250310,0.0,1,F,Aedes,steering,b1_None,1,1,Got a good one after a few attempts
561,83_20250310,1.0,2,F,Aedes,steering,b1_None,1,1,Barely on the tether; quality of recording got...
562,83_20250310,2.0,3,F,Aedes,steering,b1_None,1,1,I think this one is good
563,83_20250310,3.0,4,F,Aedes,steering,b1_None,1,1,Recording got worse over time


In [6]:
# take just the rows for trials we think we can use
log_expr_nums = log_df['Day'].str.split('_').apply(lambda row: row[0]).astype(int).values
log_axo_nums = log_df['Axo Num'].fillna(value=0).astype(int).values

good_log_idx = np.zeros((log_expr_nums.size, ), dtype=bool)
for ith, (expr_num, axo_num) in enumerate(zip(log_expr_nums, log_axo_nums)):
    if (expr_num, axo_num) in data_file_ids:
        good_log_idx[ith] = True

In [7]:
good_log_df = log_df.loc[good_log_idx]
good_log_df

,Day,Axo Num,Fly Num,Sex,Species,Target Muscle Type,Target Muscle,Electrode Num,Video Count,Notes
56,15_20240503,1.0,1,M,Aedes,power,DLM,2,1,DLM
57,15_20240503,2.0,1,M,Aedes,power,DVM,2,0,DVM
58,15_20240503,3.0,1,M,Aedes,power,DVM,3,0,DVM
61,15_20240503,6.0,3,M,Aedes,power,DLM,1,1,DLM
63,15_20240503,8.0,4,M,Aedes,power,DVM,2,0,DVM
...,...,...,...,...,...,...,...,...,...,...
559,82_20250307,9.0,4,F,Drosophila,steering,b1_None,2,0,Adjusted electrode position a tiny bit
560,83_20250310,0.0,1,F,Aedes,steering,b1_None,1,1,Got a good one after a few attempts
562,83_20250310,2.0,3,F,Aedes,steering,b1_None,1,1,I think this one is good
563,83_20250310,3.0,4,F,Aedes,steering,b1_None,1,1,Recording got worse over time


## Split multichannel rows so that we're counting each individually

In [8]:
# find multichannel rows
multichannel_idx = good_log_df['Target Muscle'].str.contains('_')

# make a new dataframe with two copies of each row
split_log_df = pd.concat(2*[good_log_df.loc[multichannel_idx]], ignore_index=True)
delete_idx = np.zeros((split_log_df.shape[0],), dtype=bool)

# loop over rows and take either the first or second Target Muscle/Electrode Num
for ith in np.arange(split_log_df.shape[0]):
    # get current target muscle and electrode nums
    target_muscle = split_log_df['Target Muscle'].loc[ith]
    electrode_num = split_log_df['Electrode Num'].loc[ith]

    if ith < (split_log_df.shape[0]/2):
        target_muscle_new = target_muscle.split('_')[0]
        try:
            electrode_num_new = str(electrode_num).split('_')[0]
        except IndexError:
            electrode_num_new = 'None'
        
    elif ith >= (split_log_df.shape[0]/2):
        target_muscle_new = target_muscle.split('_')[1]
        try:
            electrode_num_new = str(electrode_num).split('_')[1]
        except IndexError:
            electrode_num_new = 'None'

    # delete row if it's actually a single recording
    if electrode_num_new.lower() == 'none':
        delete_idx[ith] = True
        
    split_log_df.loc[ith, 'Target Muscle'] = target_muscle_new
    split_log_df.loc[ith, 'Electrode Num'] = electrode_num_new

# remove rows that don't actually correspond to a recording
split_log_df = split_log_df.loc[~delete_idx]
split_log_df

,Day,Axo Num,Fly Num,Sex,Species,Target Muscle Type,Target Muscle,Electrode Num,Video Count,Notes
0,55_20241007,1.0,1,F,Aedes,power,DVM,1,1,DLM signal large. DVM placement (EMG) is the s...
1,55_20241007,2.0,1,F,Aedes,power,DLM,2,1,DLM placement (EMG2) same as previous trial
2,55_20241007,4.0,2,F,Aedes,power,DVM,1,1,DVM placement (EMG) is the same as previous trial
3,55_20241007,5.0,2,F,Aedes,power,DLM,2,1,DLM placement (EMG2) same as previous trial
4,55_20241007,7.0,4,F,Aedes,steering,b1,1,1,Trying steering. Low cut freq to 100 Hz. Took ...
...,...,...,...,...,...,...,...,...,...,...
293,77_20250226,1.0,2,M,Aedes,power,DLM,1,0,I think this one was maybe stabbed too many ti...
294,77_20250226,2.0,2,M,Aedes,power,DLM,1,0,Just moved Target 1 a little posterior (right ...
295,77_20250226,3.0,3,M,Aedes,power,DLM,1,0,Inserting two electrodes really seems to affec...
296,77_20250226,4.0,3,M,Aedes,power,DLM,1,0,"Same positions and gains, but mosquito seems t..."


In [9]:
# merge this with the non-multichannel rows
good_log_df = pd.concat([good_log_df.loc[~multichannel_idx], split_log_df], ignore_index=True)


## Get info on species, muscle target, and electrode number

In [10]:
# make a new 'Fly ID' column that combines experiment and fly number
fly_id = good_log_df['Day'] + '_' + good_log_df['Fly Num'].astype(str)

if 'Fly ID' not in good_log_df.columns:
    good_log_df.insert(len(good_log_df.columns), 'Fly ID', fly_id)

In [11]:
# try groupby.count
good_log_df_group = good_log_df.groupby(['Species', 'Sex', 'Target Muscle', 'Fly ID']).count()
good_log_df_group

Day  Axo Num  Fly Num  \
Species    Sex Target Muscle Fly ID                                 
Aedes      F   DLM           16_20240506_1    1        1        1   
                             19_20240510_1    1        1        1   
                             19_20240510_3    1        1        1   
                             22_20240516_2    1        1        1   
                             23_20240517_1    2        2        2   
...                                         ...      ...      ...   
Drosophila M   DVM           71_20250213_2    2        2        2   
                             71_20250213_3    1        1        1   
                             72_20250214_1    1        1        1   
                             72_20250214_2    3        3        3   
                             73_20250216_1    1        1        1   

                                            Target Muscle Type  Electrode Num  \
Species    Sex Target Muscle Fly ID                                             
Aedes      F   DLM           16_20240506_1                   1              1   
                             19_20240510_1                   1              1   
                             19_20240510_3                   1              1   
                             22_20240516_2                   1              1   
                             23_20240517_1                   2              2   
...                                                        ...            ...   
Drosophila M   DVM           71_20250213_2                   2              2   
                             71_20250213_3                   1              1   
                             72_20250214_1                   1              1   
                             72_20250214_2                   3              3   
                             73_20250216_1                   1              1   

                                            Video Count  Notes  
Species    Sex Target Muscle Fly ID                             
Aedes      F   DLM           16_20240506_1            1      1  
                             19_20240510_1            1      1  
                             19_20240510_3            1      1  
                             22_20240516_2            1      0  
                             23_20240517_1            2      2  
...                                                 ...    ...  
Drosophila M   DVM           71_20250213_2            2      2  
                             71_20250213_3            1      1  
                             72_20250214_1            1      1  
                             72_20250214_2            3      3  
                             73_20250216_1            1      1  

[180 rows x 7 columns]

In [12]:
# how many unique fly IDs do we have then?
count_df = good_log_df_group.groupby(['Species', 'Sex', 'Target Muscle']).count()
count_df.drop(['Axo Num', 'Fly Num', 'Target Muscle Type', 'Electrode Num',	'Video Count', 'Notes'], axis=1, inplace=True)
count_df.rename({'Day':'Count'}, axis=1, inplace=True)
count_df

Count
Species    Sex Target Muscle       
Aedes      F   DLM               27
               DVM               28
               None               1
               b1                13
               b2                12
               b3                 2
           M   DLM               18
               DVM               15
               None               1
               b1                 1
               b2                 2
Drosophila F   DLM               19
               DVM               10
               None               1
               b1                11
           M   ?                  1
               DLM               12
               DVM                6

In [13]:
# np.savetxt('temp_log_idx.csv', good_log_idx, delimiter=',', fmt='%d')


In [14]:
# good_log_df.loc[(good_log_df['Species'] == 'Drosophila') & (good_log_df['Sex'] == 'F') & (good_log_df['Target Muscle'] == 'b1')]